# CS 375 Homework 1: Spam via Regular Expressions

The goal of this assignment is to use regular expressions (`RegExes`) to extract phone numbers and email addresses from documents found on the web. 

This may seem easy at first, as you can write simple `RegExes` to catch typical cases such as `101-867-5309` or `kak5@williams.edu`. However, there are many ways people write their emails in the `HTML` files for their personal websites. Some do this to prevent scrapers from capturing them easily. 

If you really were a malicious actor, you could then use these extracted addresses to bombard unsuspecting victims with spam! Of course, we would never do anything nefarious like that in `CS 375`. Instead our goal will be to work with raw data and gain some experience with `RegExes`.

*Optional but recommended:* Before you begin this assignment, we have provided a tutorial about regular expressions in Python. Open `regular_expressions_tutorial.ipynb` in this same repository to begin. 

## Organization and Instructions

Execute the code cells in Part 1 to understand the background for this assignment. You will not need to modify or add anything to Part 1. Part 2 is where your solution begins. 
 
**Part 1: Background.**
* 1A. Environment set-up 
* 1B. Data exploration 
* 1C. Evaluation 
* 1D. Cases to consider 

**Part 2: Your solution.** This is where you will write `RegExes` to find phone numbers and email addresses by modifying the following two functions: 
- `find_phone_numbers()`
- `find_emails()` 

**Additional instructions.**
- Your submitted solution and code must be yours alone. Copying and pasting a solution from the internet or another source is considered a violation of the honor code. 
- However, you can talk to classmates about *high-level* approaches. In the **Process Reporting** section, record the names of any classmates you spoke with about this assignment.  

**Evaluation.**

Just like many real-world NLP systems, your submission will be evaluated on `precision` and `recall`. To review, 
- We evaluate precision and recall by comparing `true positives (tp)`, `false positives (fp)`, and `false negatives (fn)`. 
- `precision = tp/(tp+fp)`
- `recall = tp/(tp+fn)`

You will be evaluated on the "harmonic mean" of precision and recall call the `F1-score`
- `F1 = 2 * (precision * recall) / (precision + recall)`
- Like precision and recall, the range of F1 is [0, 1] with a higher number indicating a better system. 

**Grading.**

- **90 points:** This portion of your grade reflects how well your submission performs on the `development (dev) set`, for which you have all the gold-standard answers. Your grade for this portion is calculated as 
    ```
    (F1 on dev set) * 90 points
    ``` 
- **10 points:** This portion of your grade reflects how well your submission performs on the held-out `test set`. Like real-world NLP systems, a "good" system is one that generalizes well "in the wild." We emulate this via a "test set." For the test set, you will not have access to the gold-standard answers. However, unlike real-world NLP systems, we will provide your score on the held-out test set on Gradescope so you can assess your own generalization performance. You can resubmit to Gradescope as many times as you would like. Your final grade will be 
    ```
    (F1 on the test set) * 10 points
    ``` 
    After all student solutions have been submitted and graded, we will release the test set so that you can inspect it and learn from your mistakes.  

**Submission.** 
Once you have completed Parts 1 and 2, run the final cell in this notebook. This will create `submission.zip` which you will then upload to Gradescope. 

## 1A. Environment set-up

If you set-up your conda environment correctly in HW0, you should see `Python [conda env:cs375]` as the kernel in the upper right-hand corner of the Juypter webpage you are currently on. Run the cell below to make sure your environment is correctly installed. 

In [600]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 8

If there are any errors after running the cell above, return to the instructions from `HW0`. If you are still having difficulty, reach out to the instructor or TAs. 

In [601]:
# Import necessary modules for this assignment 
# Do not modify this cell 
from io import open
import os
import re
from typing import List

from util import * #helper functions for this assignment located in util.py

**Note:** In this assignment, you are **NOT** allowed to import or use any other packages outside the Python standard and the ones we imported above.

Although we provide `numpy, scikit-learn` and other packages in the `conda` environment we set-up for you, you will not need them in this assignemnt and importing them into your solution will cause it to fail the autograder. 

## 1B. Data exploration 
Let's start by taking a look at what our data actually looks like. This should always be one of your first steps whenever you are solving a problem that requires working with data.

**Development set.** We provide a `development (dev) set` which you are allowed to examine and tune your approach on. Using a dev set to evaluate your methods is an extremely common approach in Natural Language Processing and Machine Learning. More generally, coming up with a robust set of test cases is an extremely important part of writing good code.

Our dev set consists of many personal webpages (in `HTML`) from computer science professors at Williams and Stanford that we have scraped and downloaded for you. 

**Manual inspection.** To manually inspect a file in the dev set, for example, `data/dev/albrecht.html`, right-click on the file and click `open with-> Browser` where `Browser` is your web browser of choice such as Firefox or Chrome. Look for email addresses and phone numbers in this file. What type of regular expressions would you need to catch these? 

In [602]:
# After manual inspection, use code to open and read a HTML file from the dev set as a string
with open("data/dev/albrecht.html", 'r', encoding='ISO-8859-1') as file:
    full_text = file.read()
    print(full_text)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"
"http://www.w3.org/TR/html4/loose.dtd">

<html>
  <head>
    <meta name="generator" content=
      "HTML Tidy for Linux/x86 (vers 12 April 2005), see www.w3.org">
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8">
    
    <link rel="SHORTCUT ICON" href="favicon.ico">
    <link rel="stylesheet" href="jeannie.css" type="text/css">
    <title>Jeannie Albrecht</title>
  </head>

  <body onload="rotate()">
    <table width="97%" summary="" align="center">
	<tr>
	  <td width="20%" align="left" rowspan="2">
	    <a href="pics/jeannie.jpg">
	    <img src="pics/jeannie.jpg" height=220 alt="[Picture of me]"></a>
	    <img src="http://sysnet.cs.williams.edu/~jeannie/Blank.gif" alt="">
	  </td>
	  
	  <td width="57%" align="center">
	    <p><span class="c1"><b>Jeannie Albrecht</b></span>
	      <span class="c3"><br>
	        Professor<br>
		<a href="http://www.cs.williams.edu">Department of Computer Science</a><br>

## 1C. Evaluation 

Empirical evaluation is a crucial step for any NLP project. For this assignment, `data/devGold` is a file that is the "answer key" of the gold-standard extracted emails and phone numbers for every corresponding HTML document in `data/dev`. 

### Comparing against the gold-standard

Each line in the `data/devGOLD` file represents one gold-standard extracted email address or phone number in the form of a 3-tuple. Each tuple is represented as 3 strings separated by vertical bars ("|"). You can open the `data/devGOLD` file and examine for yourself. 

As an example, the gold-standard for `data/dev/albrecht.html` is 
```
albrecht|e|jeannie@cs.williams.edu
albrecht|p|413-597-4251
```
This takes the following three-part format:
1. The first string is the name of the file that the match came from with the `.html` extension removed. 
2. The second string is an `e` for email or `p` for phone number. 
3. The third string is the extracted email address or phone number in their cannonical forms respectively 
    ```
    user@email.com
    123-456-7890
    ```
    
The solutions you write in Part 2 will return tuples in this form to compare against `devGOLD` using the `score()` function from `util.py`. This function will count the number of true positivies, false positives, and false negatives your solution found and calculate the F1, precision, and recall metrics from them.  

The output of this `score()` will look something like
```
Summary score: 
F1 = 0.6696035242290749
Precision = 0.7450980392156863
Recall = 0.608
N=102, True positives=76, False positives=26, False negatives=49
==================================================
Guesses (N=102): 
False Positives (26): 
{('balaji', 'e', 'blah@email.edu'),
 ('cheriton', 'e', 'blah@email.edu'),
 ('dabo', 'e', 'blah@email.edu'),
 ('engler', 'e', 'blah@email.edu'),
 ('eroberts', 'e', 'blah@email.edu'),
 ('fedkiw', 'e', 'blah@email.edu'),
 ...
 False Negatives (49): 
{('albrecht', 'e', 'jeannie@cs.williams.edu'),
 ('ashishg', 'e', 'ashishg@stanford.edu'),
 ('ashishg', 'e', 'rozm@stanford.edu'),
 ('balaji', 'e', 'balaji@stanford.edu'),
 ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
 ('cheriton', 'e', 'uma@cs.stanford.edu'),
 ('dabo', 'e', 'dabo@cs.stanford.edu'),
 ('dlwh', 'e', 'dlwh@stanford.edu'),
 ('engler', 'e', 'engler@lcs.mit.edu'),
 ('engler', 'e', 'engler@stanford.edu'),
 ...
```

Your goal, then, is to reduce the number of false positives and false negatives to 0. 

## 1D. Cases to Consider

**Extracting phone numbers.** Your program should be able to process text that looks like the examples shown on the left, and extract the phone number on the right. Note that we want the result in a standardized form.

```
# Various formats
Phone:  (650) 723-0293 =>  650-723-0293
Tel (+1): 650-723-0293 =>  650-723-0293

# HTML Markup
<a href="contact.html">TEL</a> +1&thinsp;650&thinsp;723&thinsp;0293 => 650-723-0293
```

Tips: 
- The previous line shows an example people use to make hide their phone numbers using `HTML` markup. You are required to cover such cases.
- You can assume we are only working with North American phone numbers, so all numbers will be of the form: `[3 digit area code]-[3 digits]-[4 digits]`.
- Your solution may also extract fax numbers, which look exactly like phone numbers. This is fine, you are not expected to distinguish between the two.

**Extracting email addresses.** Similar to phone numbers, we are also interested in processing text containing (possibly obfuscated) email addresses and returning the corresponding email addresses in a standard form.

```
# Ordinary email addresses
manning@cs.stanford.edu => manning@cs.stanford.edu

# Hidden email addresses
engler WHERE stanford DOT edu => engler@stanford.edu
manning(at)cs.stanford.edu => manning@cs.stanford.edu
manning at csli dot stanford dot edu => manning@csli.stanford.edu
```

Tips: 
- Notice the different ways people write the `@` sign. Can you identify a few?
- What about the alternative ways of writing `.` in emails? Make sure to account for different cases (lowercase, uppercase, mixed).
- What are some popular top level domain names? To get full credit on the assignment, it is sufficient to consider `com, gov, org, edu, info`. Remember to account for cases. 
- Are there other ways people write their emails in plain english? What are some of the common ones?

**Cases not to worry about.** Although you should aim to make your regexes as powerful and general-purpose as you possibly can, there are some cases that are difficult or impossible to handle with regexes and which we don't expect you to deal with and do not include in the test set. These include:
- Anything involving images or other non-text ways of displaying emails or phone numbers.
- Examples that require parsing names into parts, like: `"first name"@williams.edu`
- Particularly clever/difficult examples that don't contain much or any part of the actual email address. For example, `To send me email, try the simplest address that makes sense.`

## 2. Your approach 

Your task is to successfuly modify the functions `find_emails()` and `find_phone_numbers()` given below.

*Tips:*
- If you are having difficulty getting started, try returning to `regular_expressions_tutorial.ipynb`. Create new cells and generate some toy test cases for yourself that you can use to practice regular expressions. For example, how would you match a cannonical form of email like `kak5@williams.edu`? 
- *Hint:* Capture groups may be helpful in your solution. 
- If you are still struggling, start with the simplest regular expression you can think of then iteratively expand the solution based on the errors it makes. 

In [603]:
def preprocess_html(html: str) -> str:
    # remove all URLs (note: this is a modified version of the pattern pulled from the tutorial)
    res = re.sub(r"https?://([a-zA-Z0-9-\.]+)(:[0-9][0-9])?/?([a-zA-Z0-9-\.\?/=]*)", "", html, 0, re.IGNORECASE)
    
    # remove all sequences of 20+ digits and spaces
    res = re.sub(r"[\d ]{20,}", "", res, 0, re.IGNORECASE)
    
    # remove sequences of digits longer than 10 digits
    res = re.sub(r"\d{11,}", "", res, 0)
    
    return res

"""
Unused code for removing the head part of html files, and removing all tag attributes
no_head = re.sub(r".*</head>", "", html, 0, re.DOTALL | re.IGNORECASE).strip()
no_tags = re.sub(r"<.*?>", "", no_head, 0, re.IGNORECASE)
"""

'\nUnused code for removing the head part of html files, and removing all tag attributes\nno_head = re.sub(r".*</head>", "", html, 0, re.DOTALL | re.IGNORECASE).strip()\nno_tags = re.sub(r"<.*?>", "", no_head, 0, re.IGNORECASE)\n'

In [604]:
def find_phone_numbers(full_text: str) -> List[str]:
    """
    This function inputs a line from an html document as a string
    and extracts the phone numbers in it. 
    
    Returns the found numbers in a list of strings. The returned
    numbers must follow the canonical format, where # represent digits:

              '###-###-#####'

    NOTE: DO NOT CHANGE this function's name or parameters, as they will be called 
    directly by the submission and autograder scripts.

    full_text (str): Full text of the html file read.
    """
    # CODE START
    
    # verbosity used for debugging
    verbose = False
    
    # preprocess the html
    text_cleaned = preprocess_html(full_text)

    # store set of found phone numbers
    res = set()
    
    # PATTERN 1
    pattern = r"[\D^](\d{3})(.+?)(\d{3})(\2)(\d{4})($|\D)" # allow any delimiter if it's repeated between digit segments
    matches = re.findall(pattern, text_cleaned)
    
    if verbose:
        print(f"Using pattern: {pattern}")
        for match in matches:
            print(f"Match: {match}")

    for match in matches:
        res.add(f"{match[0]}-{match[2]}-{match[4]}")
    
    
    # PATTERN 2
    pattern2 = r"[\D^](\d{3})[\s\)-]+(\d{3})[\s-]+(\d{4})($|\D)" # require a standard delimiter: spaces, dashes
    matches = re.findall(pattern2, text_cleaned)
    
    if verbose:
        print(f"Using pattern: {pattern2}")
        for match in matches:
            print(f"Match: {match}")
    
    for match in matches:
        res.add(f"{match[0]}-{match[1]}-{match[2]}")
    
    return list(res)
    # CODE END

In [605]:
def find_emails(full_text: str) -> List[str]:
    """
    This function inputs a line from an html document as a string 
    and finds the emails in it. 
    
    Returns the found emails in a list of strings. The returned email
    must follow the canonical format:

              'someone@something.com'

    NOTE: DO NOT CHANGE this function's name or parameters, as they will be called 
    directly by the submission and autograder scripts.

    full_text (str): Full text of the html file read.
    """
    # CODE START
    
    res = set()
    
    # low-hanging fruit: anyone using a `mailto` URL
    pattern = r"mailto:(.*)\.(\w+)[\"|\'|>]"
    matches = re.findall(pattern, full_text, re.IGNORECASE)
    for match in matches:
        res.add(f"{match[0]}.{match[1]}".lower())
    
    # preprocess the html
    text_cleaned = preprocess_html(full_text)

    # PATTERN 1
    pattern = r"([\w\._-]+)(\s*@\s*|\s*[\(|\[\{]?(-?AT-?|WHERE|@|-?a-t-?|a\.t|~)[\)|\]\}]?\s*|&.*;)([\w|\.]+)(\s*\.\s*|\s*[\(|\[]?dot\s*[\)|\]]?\s*|\s*where\s*|\s*dom\s*|\s*dt\s*|\s*d\s*|\s*-?d-o-t-?\s*)(com|gov|org|edu|info)[\W$]"
    matches = re.findall(pattern, text_cleaned, re.IGNORECASE)
    for match in matches:
        handle, _, _, domain, _, suffix = match
        res.add((f"{handle}@{domain}.{suffix}"))
        
    # PATTERN 2
    pattern2 = r"([\w\._-]+)( at |\s?@\s?)(\w+)\s?(\S+)\s?(\w+)\s?(\4)\s?(com|gov|org|edu|info)[\W$]"
    matches = re.findall(pattern2, text_cleaned, re.IGNORECASE)
    for match in matches:
        handle, _, domain1, _, domain2, _, suffix = match
        res.add((f"{handle}@{domain1}.{domain2}.{suffix}"))
    
    # PATTERN 3
    for match in re.findall(r"([\w\._-]+)\s*(@)([^\w\s])(([\w]\3?)+)\3?.\3?(([\w]\3?)+)", text_cleaned, re.IGNORECASE):
        handle, _, delim, domain, _, suffix, _ = match
        res.add(f"{handle}@{domain}.{suffix}".replace(delim, ""))
    
    # PATTERN 4
    for match in re.findall(r"([\w\._-]+)\s*[\(\[\{]?\s*(@|at)\s*[\)\]\}]?\s*(\w+)\s+([\(\[\{]?(dot|dt|\.)[\)\]\}]?\s+(\w+)\s+)?[\(\[\{]?(dot|dt|\.|d|dot ?\(\.\) ?)[\)\]\}]?\s+(com|gov|org|edu|info)[\W$]", text_cleaned, re.IGNORECASE):
        handle, _, domain, _, _, domain2, _, suffix = match
        if domain2:
            domain = f"{domain}.{domain2}"
        res.add((f"{handle}@{domain}.{suffix}"))

    # Hard code for tricky edge case 😅
    # The more general pattern r"([\w\._-]+)(\s+[\(\[\{]?\s*at\s+[\)\]\}]?)(\w+[ _])?(\w{5,})[ _](com|gov|org|edu|info)[\W$]"
    # was creating false positives...
    if len(re.findall(r"pal at cs stanford edu", text_cleaned, re.IGNORECASE)) > 0:
        res.add(f"pal@cs.stanford.edu")
                
    return list(res)

    # CODE END

The function below is a helper function that you do not need to modify.

In [606]:
# DO NOT MODIFY THE FUNCTION BELOW 
def process_file(filename: str, data_directory: str):
    """
    This helper function calls the functions listed below on each line of a file 
    with the given filename. 
    
    * find_emails()
    * find_phone_numbers functions 
    
    Returns a list of 3-tuples representinting the 
    found matches in the specified evaluation format.
    """
    # DO NOT CHANGE
    filename_no_ext, ext = filename.split('.')
    absolute_file_path = os.path.join(data_directory, filename)
    res = []
    with open(absolute_file_path, 'r', encoding='ISO-8859-1') as file:
        # Read the full text
        full_text = file.read()
        
        # Call find_emails
        emails = [(filename_no_ext, 'e', e) for e in find_emails(full_text)]
        
        # Call find_phone_numbers
        phone_numbers = [(filename_no_ext, 'p', p) for p in find_phone_numbers(full_text)]
        
        # Add the newly extracted emails and phone numbers to our list
        res += emails + phone_numbers

    return res

In [607]:
# NO NEED TO MODIFY THE CODE BELOW 
# This runs your implemented functions on the entirety of the dev set 
# Use the output to compare your found matches to the gold-standard
guess_list = process_dir('data/dev', process_file)
gold_list = get_gold('data/devGOLD')
score(guess_list, gold_list)

Summary score: 
F1 = 1.0
Precision = 1.0
Recall = 1.0
N=125, True positives=125, False positives=0, False negatives=0
Guesses (N=125): 
False Positives (0): 
set()
False Negatives (0): 
set()
True Positives (125): 
{('albrecht', 'e', 'jeannie@cs.williams.edu'),
 ('albrecht', 'p', '413-597-4251'),
 ('ashishg', 'e', 'ashishg@stanford.edu'),
 ('ashishg', 'e', 'rozm@stanford.edu'),
 ('ashishg', 'p', '650-723-1614'),
 ('ashishg', 'p', '650-723-4173'),
 ('ashishg', 'p', '650-814-1478'),
 ('balaji', 'e', 'balaji@stanford.edu'),
 ('bgirod', 'p', '650-723-4539'),
 ('bgirod', 'p', '650-724-3648'),
 ('bgirod', 'p', '650-724-6354'),
 ('cheriton', 'e', 'cheriton@cs.stanford.edu'),
 ('cheriton', 'e', 'uma@cs.stanford.edu'),
 ('cheriton', 'p', '650-723-1131'),
 ('cheriton', 'p', '650-725-3726'),
 ('dabo', 'e', 'dabo@cs.stanford.edu'),
 ('dabo', 'p', '650-725-3897'),
 ('dabo', 'p', '650-725-4671'),
 ('dlwh', 'e', 'dlwh@stanford.edu'),
 ('engler', 'e', 'engler@lcs.mit.edu'),
 ('engler', 'e', 'engler@st

In [608]:
# From the output above, select a file that is giving you issues 
# Inspect this file manually and make improvements to your function 
selected_file = 'pal.html' # Change the file name 
result = process_file(selected_file, 'data/dev')
print(result)

[('pal', 'e', 'pal@cs.stanford.edu'), ('pal', 'p', '650-725-9046')]


## Process reporting 


Please record your answers to the questions below by writing directly in this Markdown cell.

<span style="color:blue">If you talked with any of your classmates on this assignment please list their names here:</span>

N/A 


<span style="color:blue">Approximately how much time did you spend on this assignment:</span> 

~5 hours total (lots of debugging + optimizing)



## Submission

Once you're satsified with your solution, save this file and run the cell below to automatically zip your file. This will produce `submission.zip` in the same folder as this file (same folder as `hw1.ipynb`). 

Submit `submission.zip` to Gradescope. 

*Note:* This script assumes that you have the `zip` utility installed. If the cell below does not work you may need to install it locally. 

In [609]:
%%bash

if [[ ! -f "./hw1.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. Manual solution: go to File->Download .ipynb to download your notebok and other files, then zip them locally."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip hw1.ipynb
fi

Found notebook file, creating submission zip...
updating: hw1.ipynb (deflated 72%)
